In [2]:
import os
import sys
import json
import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from contextlib import redirect_stdout
import numpy as np

# ----- Define Tee class for logging -----
class Tee(object):
    def __init__(self, *fileobjects):
        self.fileobjects = fileobjects
    def write(self, text):
        for f in self.fileobjects:
            f.write(text)
            f.flush()
    def flush(self):
        for f in self.fileobjects:
            f.flush()

# ----- Specify paths for saving logs, metrics, and models -----
log_file_path = "./logs/cifar10_training_log.txt"
metrics_file_path = "./metrics/cifar10_training_metrics.json"
best_model_path = "./models/cifar10_best_student_model.pth"
save_path = "./models/cifar10_student_model"
teacher_checkpoint = "/notebooks/Resnet18/models/cifar10_best_model"  # Pre-trained teacher

# ----- Define a simple transform for training, validation, and testing -----
# No data augmentation here—just tensor conversion and normalization.
norm = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    norm
])

# ----- Load CIFAR-10 datasets without additional augmentations -----
# For training and validation, split the training set.
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=eval_transform)
train_size = int(0.8 * len(full_trainset))
val_size = len(full_trainset) - train_size
trainset, valset = random_split(full_trainset, [train_size, val_size])

# For testing, use the standard CIFAR-10 test set.
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=eval_transform)

# Create DataLoaders (no mixup collate function is needed here).
train_loader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)
val_loader = DataLoader(valset, batch_size=256, shuffle=False, num_workers=2)
test_loader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

# ----- Prepare Teacher and Student Models -----
# Teacher: ResNet18 modified for CIFAR-10.
teacher = models.resnet18(weights=None)
teacher.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
teacher.maxpool = nn.Identity()
num_ftrs = teacher.fc.in_features
teacher.fc = nn.Linear(num_ftrs, 10)

# Student: SqueezeNet (using squeezenet1_1) modified for CIFAR-10.
student = models.squeezenet1_1(pretrained=False)
# Adjust the classifier for 10 classes.
student.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
student.num_classes = 10

# ----- Device and Checkpoints Setup -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 200
learning_rate = 0.001
checkpoint_frequency = 10
start_epoch = 0
start_checkpoint = None  # Optionally, resume student training checkpoint

# Load teacher checkpoint (pre-trained on CIFAR-10) if available.
if os.path.exists(teacher_checkpoint):
    teacher_state = torch.load(teacher_checkpoint, map_location=device)
    teacher.load_state_dict(teacher_state)
else:
    print("Warning: Teacher checkpoint not found. Teacher model may be untrained.")

teacher.to(device)
teacher.eval()  # Teacher is fixed during distillation.
for param in teacher.parameters():
    param.requires_grad = False

# Optionally resume student training from a checkpoint.
if start_checkpoint:
    student_state = torch.load(start_checkpoint, map_location=device)
    student.load_state_dict(student_state)
student.to(device)

# ----- Setup Loss Functions and Optimizer -----
# Using KL divergence as the distillation loss.
kl_loss = nn.KLDivLoss(reduction='batchmean')
# For evaluation, we use standard cross-entropy.
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
temperature = 4.0

# ----- Initialize Metrics Storage -----
metrics = {
    "epochs": [],
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}
best_val_acc = 0.0
best_epoch = None

# ----- Setup Logging (Tee and TensorBoard) -----
log_capture = io.StringIO()
tee = Tee(sys.stdout, log_capture)  # Prints and captures output
writer = SummaryWriter(log_dir='./runs/cifar10_naive_distillation_experiment')

with redirect_stdout(tee):
    print("Training student (SqueezeNet) with teacher (ResNet18) naive knowledge distillation on CIFAR-10...")
    for epoch in range(start_epoch, start_epoch + num_epochs):
        student.train()
        running_loss = 0.0
        total = 0
        correct = 0
        
        # Save a checkpoint at the specified frequency.
        if epoch % checkpoint_frequency == 0:
            torch.save(student.state_dict(), f"{save_path}_{epoch}.pth")
        
        # Training loop (no mixup, plain inputs and labels)
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            # Get teacher predictions without backprop.
            with torch.no_grad():
                teacher_logits = teacher(inputs)
            # Get student predictions.
            student_logits = student(inputs)
            
            # Compute the distillation loss (temperature-scaled KL divergence).
            teacher_soft = torch.softmax(teacher_logits / temperature, dim=1)
            student_log_soft = torch.log_softmax(student_logits / temperature, dim=1)
            loss_distill = (temperature * temperature) * kl_loss(student_log_soft, teacher_soft)
            
            # For naive knowledge distillation, we only use the distillation loss.
            loss = loss_distill
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            total += inputs.size(0)
            # Calculate training accuracy using hard predictions.
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
        
        train_epoch_loss = running_loss / total
        train_epoch_acc = 100. * correct / total

        # ----- Validate on the Validation Set -----
        student.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = criterion(outputs, labels)  # Standard CE loss for evaluation.
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        val_epoch_loss = val_loss / val_total
        val_epoch_acc = 100. * val_correct / val_total

        metrics["epochs"].append(epoch + 1)
        metrics["train_loss"].append(train_epoch_loss)
        metrics["train_acc"].append(train_epoch_acc)
        metrics["val_loss"].append(val_epoch_loss)
        metrics["val_acc"].append(val_epoch_acc)

        print(f"Epoch [{epoch+1}/{start_epoch + num_epochs}] - Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.2f}% | Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%")
        
        # TensorBoard logging.
        writer.add_scalar('Loss/Train', train_epoch_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_epoch_acc, epoch)
        writer.add_scalar('Loss/Validation', val_epoch_loss, epoch)
        writer.add_scalar('Accuracy/Validation', val_epoch_acc, epoch)
        
        # Save the best model based on validation accuracy.
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_epoch = epoch + 1
            torch.save(student.state_dict(), best_model_path)
            print(f"New best student model found at epoch {epoch+1} with Val Acc: {val_epoch_acc:.2f}%")
        
        # Save metrics JSON after each epoch.
        with open(metrics_file_path, "w") as f:
            json.dump(metrics, f, indent=4)
    
    print("Training complete.")
    torch.save(student.state_dict(), save_path + "_final.pth")

# Close the TensorBoard writer.
writer.close()

# Write captured logs to the specified log file.
with open(log_file_path, "w") as f:
    f.write(log_capture.getvalue())

print("Training log captured and saved to", log_file_path)
print("Training metrics saved to", metrics_file_path)
print("Best student model saved from epoch", best_epoch, "with validation accuracy of", best_val_acc)


Files already downloaded and verified
Files already downloaded and verified
Training student (SqueezeNet) with teacher (ResNet18) naive knowledge distillation on CIFAR-10...


Epoch 1/200: 100%|██████████| 157/157 [00:07<00:00, 22.23it/s]


Epoch [1/200] - Train Loss: 15.1668, Train Acc: 13.23% | Val Loss: 2.3742, Val Acc: 20.37%
New best student model found at epoch 1 with Val Acc: 20.37%


Epoch 2/200: 100%|██████████| 157/157 [00:04<00:00, 31.66it/s]


Epoch [2/200] - Train Loss: 12.9461, Train Acc: 23.13% | Val Loss: 2.0687, Val Acc: 30.04%
New best student model found at epoch 2 with Val Acc: 30.04%


...

Epoch [198/200] - Train Loss: 0.8599, Train Acc: 93.84% | Val Loss: 2.0516, Val Acc: 66.48%


Epoch 199/200: 100%|██████████| 157/157 [00:05<00:00, 30.62it/s]


Epoch [199/200] - Train Loss: 0.8553, Train Acc: 93.91% | Val Loss: 2.1134, Val Acc: 66.56%


Epoch 200/200: 100%|██████████| 157/157 [00:05<00:00, 30.81it/s]


Epoch [200/200] - Train Loss: 0.8532, Train Acc: 93.82% | Val Loss: 2.1743, Val Acc: 66.02%
Training complete.
Training log captured and saved to ./logs/cifar10_training_log.txt
Training metrics saved to ./metrics/cifar10_training_metrics.json
Best student model saved from epoch 89 with validation accuracy of 68.07
